In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import catboost
import argparse
import os
from math import sqrt

import numpy as np
import pandas as pd
from functools import reduce
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from statistics import mean
import GPyOpt
import time
from itertools import chain

In [ ]:
def load_data(input_file, categories, frequency):

    m3c_file = pd.ExcelFile(input_file)

    sheet_name = 'M3' + frequency

    m3c_month_df = m3c_file.parse(sheet_name)

    # strip unnecessary spaces from 'Category' column
    m3c_month_df['Category'] = m3c_month_df['Category'].apply(lambda x: x.strip())

    return m3c_month_df[m3c_month_df['Category'].isin(categories)]

In [ ]:
def next_series_generator(data_df, test_data_size):

    scaler = preprocessing.MinMaxScaler()

    for _, row in data_df.iterrows():
        series_size = row['N']
        start_year = row.iloc[4]
        start_month = row.iloc[5]
        row = row.iloc[6:series_size+6]
        date = None
        if frequency == 'Year':
            freq = 'A'
        elif frequency == 'Month':
            if start_year == 0 or start_year == '0':
                start_year = 1976
                start_month = 1
            freq = 'M'
        elif frequency == 'Quart':
            freq = 'Q'
        else:
            date = range(series_size)
        if not date:
            date = pd.date_range('1/{}/{}'.format(start_month, start_year), periods=series_size, freq=freq)
        row.index = date
        
        train_data = row.iloc[range(series_size - test_data_size)]
        test_data = row.iloc[range(series_size - test_data_size, series_size)]
        # normalise data to range [0, 1]
        train_tmp = train_data.values.astype(np.float64).reshape(-1,1)
        test_tmp = test_data.values.astype(np.float64).reshape(-1, 1)
        scaler.fit(train_tmp)
        train_data = pd.Series([el[0] for el in scaler.transform(train_tmp).tolist()],  index = train_data.index)
        test_data = pd.Series([el[0] for el in scaler.transform(test_tmp).tolist()],  index = test_data.index)

        yield train_data, test_data, scaler

In [ ]:
def extract_features(df):
    features = pd.DataFrame()
    if frequency == 'Other':
        features['index'] = df.index
        return features
    features['Year'] = df.index.year
    if frequency == 'Month':
        features['Month'] = df.index.month
        features['Quarter'] = df.index.quarter
    elif frequency == 'Quart':
        features['Quarter'] = df.index.quarter
    return features

In [ ]:
def time_series_cross_validation(model, train_data):
    """Perform cross validation of time series and return mean of rmse of all models created for validation"""
    fold_nr = 0
    rmse_per_fold = []
    max_nr_of_folds = 6
    train_data_size = len(train_data)

    # set window_size so that max_nr_of_folds validations can be done
    # and model is fitted on at least half of train data
    window_size = int(train_data_size / 2 / max_nr_of_folds) or 1
    window_slide = window_size

    while fold_nr < max_nr_of_folds:
        index = train_data_size - window_size - window_slide * fold_nr
        # at least as much fit data as window_size
        if index < window_size:
            break
        fit_data = train_data.iloc[:index]
        validation_data = train_data.iloc[index:index + window_size]
        model = fit_model(model, fit_data)

        validation_forecast = model.predict(extract_features(validation_data))
        rmse_per_fold.append(sqrt(mean_squared_error(validation_data.values, validation_forecast)))
        fold_nr += 1
    return mean(rmse_per_fold), model

def fit_model(model, fit_data):
    model = model.fit(extract_features(fit_data), fit_data.values, verbose=None, logging_level='Silent')
    return model

def make_prediction(model, train_data, test_data):
    # prediction of train data
    prediction = model.predict(list(range(len(train_data))))
    # prediction of test data
    forecast = model.predict(list(range(len(test_data))))
    return [list(prediction), list(forecast)]

In [ ]:
bounds = [
    {'name': 'learning_rate', 'type': 'continuous', 'domain': (0.01, 0.8)},
    {'name': 'max_depth', 'type': 'discrete', 'domain': (2, 10)},
    {'name': 'colsample_bylevel', 'type': 'continuous', 'domain': (0.5, 1.0)},
    {'name': 'bagging_temperature', 'type': 'continuous', 'domain': (0.0, 100.0)},
    {'name': 'random_strength', 'type': 'continuous', 'domain': (0.0, 100.0)},
    {'name': 'reg_lambda', 'type': 'continuous', 'domain': (1.0, 100.0)},
]

def find_parameters(train_data):
    def f(x):
        model = catboost.CatBoostRegressor(
            iterations=300,
            **{el['name']: float(x[:, n]) if el['type'] == 'continuous' else int(x[:,n]) for n, el in enumerate(bounds)}
        )
        score, model = time_series_cross_validation(model, train_data=train_data)
        return -score
    
    myBopt = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)
    myBopt.run_optimization(max_iter=5)
    
    return myBopt

In [ ]:
times = []
for frequency in ['Year', 'Month','Quart', 'Other']:
    for category in ['MICRO', 'INDUSTRY', 'MACRO', 'FINANCE', 'DEMOGRAPHIC', 'OTHER']:
        times.append((frequency, category, 'start', time.time()))
        dfa = load_data('data/M3C.xls', [category], frequency)
        if frequency == 'Year':
            test_data_size = 6
        elif frequency == 'Quart':
            test_data_size = 8
        elif frequency == 'Month':
            test_data_size = 18
        else:
            test_data_size = 8

        predicted_data_matrix = []
        expected_data_matrix = []

        normalized_forecast_data_matrix = []
        normalized_test_data_matrix = []

        # list of dicts of parameters of each trained model
        parameters_dicts = []

        rmse_prediction = []
        rmse_forecast = []
        rmse_total = []
        print(times[-1])
        for n, (train_data, test_data, scaler) in enumerate(next_series_generator(dfa, test_data_size)):
            myopt = find_parameters(train_data)
            model = catboost.CatBoostRegressor(**{el['name']: 
                                                  float(myopt.x_opt[n]) if el['type'] == 'continuous' 
                                                                        else int(myopt.x_opt[n]) 
                                                  for n, el in enumerate(bounds)})
            model = fit_model(model, train_data)
            prediction, forecast = model.predict(extract_features(train_data)), model.predict(extract_features(test_data))
            parameters_dicts.append(model.get_params())
#             print forecast
            predicted_data = scaler.inverse_transform([list(prediction) + list(forecast)]).tolist()[0]
            expected_data = scaler.inverse_transform([list(train_data) + list(test_data)]).tolist()[0]

            rmse_prediction.append(sqrt(mean_squared_error(train_data, prediction)))
            rmse_forecast.append(sqrt(mean_squared_error(test_data, forecast)))
            rmse_total.append(sqrt(mean_squared_error(list(train_data) + list(test_data), list(prediction) + list(forecast))))

            normalized_forecast_data_matrix.append(forecast)
            normalized_test_data_matrix.append(test_data.values)

            predicted_data_matrix.append(predicted_data)
            expected_data_matrix.append(expected_data)
        if not predicted_data_matrix:
            continue
            
        times.append((frequency, category, 'stop', time.time()))
        data_columns = list(range(1, max(len(x) for x in predicted_data_matrix) + 1))
        series_index = list(range(len(predicted_data_matrix)))
        series_length = [len(x) for x in predicted_data_matrix]

        predicted_data_df = pd.DataFrame(predicted_data_matrix, columns=data_columns)
        predicted_data_df['series'] = series_index
        predicted_data_df['N'] = series_length
        predicted_data_df['NF'] = test_data_size

        # take parameters names from first dict
        parameters_names = list(parameters_dicts[0].keys())
        for parameter in parameters_names:
            predicted_data_df[parameter] = [x[parameter] for x in parameters_dicts]

        predicted_data_df['rmse_prediction'] = rmse_prediction
        predicted_data_df['rmse_forecast'] = rmse_forecast
        predicted_data_df['rmse_total'] = rmse_total

        # change the order of columns
        output_columns = ['series', 'N', 'NF'] + parameters_names + ['rmse_prediction', 'rmse_forecast', 'rmse_total'] \
                         + data_columns
        predicted_data_df = predicted_data_df[output_columns]

        expected_data_df = pd.DataFrame(expected_data_matrix, columns=data_columns)
        expected_data_df['series'] = series_index
        expected_data_df['N'] = series_length
        expected_data_df['NF'] = test_data_size

        output_columns = ['series', 'N', 'NF'] + data_columns
        expected_data_df = expected_data_df[output_columns]

        # calculate error separately for each month of forecast
        rmse_per_month = []
        for i in range(test_data_size):
            month_forecast = [x[i] for x in normalized_forecast_data_matrix]
            month_expected = [x[i] for x in normalized_test_data_matrix]
            rmse_per_month.append(sqrt(mean_squared_error(month_expected, month_forecast)))

        # calculate total error of all forecasts
        all_forecast = list(chain(*normalized_forecast_data_matrix))
        all_test = list(chain(*normalized_test_data_matrix))
        total_rmse = sqrt(mean_squared_error(all_test, all_forecast))

        output_dir = '{frequency}_{category}'.format(frequency=frequency, category=category)
        if not os.path.isdir(output_dir):
            os.makedirs(output_dir)

        normalized_forecast_data_df = pd.DataFrame(normalized_forecast_data_matrix)
        normalized_test_dat_df = pd.DataFrame(normalized_test_data_matrix)

        normalized_forecast_data_df.to_csv(os.path.join(output_dir, 'normalized_forecast.tsv'), sep='\t', index=False)
        normalized_test_dat_df.to_csv(os.path.join(output_dir, 'normalized_test_data.tsv'), sep='\t', index=False)

        predicted_data_df.to_csv(os.path.join(output_dir, 'predictions.tsv'), sep='\t', index=False)
        expected_data_df.to_csv(os.path.join(output_dir, 'expected.tsv'), sep='\t', index=False)

        with open(os.path.join(output_dir, 'rmse_file.tsv'), 'w') as f:
            f.write('\t'.join([str(x) for x in rmse_per_month]))
            f.write('\n' + str(total_rmse))
times